In [1]:
import matplotlib.pyplot as plt
import numpy as np
import math
from valueMaps import ValueMap

In [2]:
def scaleDownArray(array):
    offsets = [-1,0,1]
    
    smallArray = np.empty((round(array.shape[0]/3), round(array.shape[1]/3)))
    largeArray = array.copy()
    for rowIndex in range(math.floor(largeArray.shape[0] / 3)):
        for columnIndex in range(math.floor(largeArray.shape[1]/3)):
            val = 0
            for i in offsets:
                for j in offsets:
                    val = val + largeArray[(rowIndex)*3+i+1][(columnIndex)*3+j+1]
            smallArray[rowIndex][columnIndex] = val
    return smallArray

In [3]:
# Make maptlotlib show graphs in a new window
%matplotlib qt 

In [4]:
dataDate = "10/1/2024" # Date of the data to be collected in format %D/%M/%Y

gridData = []

In [5]:
# path = f"data/solarIradiance/{dataDate.replace('/', '')}.grid"
# path = "data/density.grid"
path = "data/compressedDensity.grid"
with open(path, "r", encoding="utf-8") as gridFile:
    gridData = gridFile.readlines()
    
for index, line in enumerate(gridData):
    gridData[index] = line.replace("\n", "")

In [6]:
gridSize = [-1,-1]
southWestPos = [-1,1]
cellSize = -1
noDataVal = -1
for line in gridData:
    if "ncols" in line:
        gridSize[1] = int(line[line.find(" ")+1:])
    if "nrows" in line:
        gridSize[0] = int(line[line.find(" ")+1:])
    if "cellsize" in line:
        cellSize = float(line[line.find(" ")+1:])
    if "nodata_value" in line:
        noDataVal = float(line[line.find(" ")+1:])
    if "SOUTHWEST LATITUDE" in line:
        line = line.replace(" ", "")
        southWestPos[0] = float(line[line.find("LATITUDE")+8:])
    if "SOUTHWEST LONGITUDE" in line:
        line = line.replace(" ", "")
        southWestPos[1] = float(line[line.find("LONGITUDE")+9:])

        

In [17]:
# Put grid data into a np array
mapData = np.empty((gridSize[0], gridSize[1]))

rowIndex = 0 
for line in gridData:
    row = line.split(" ")
    try:
        testVal = float(row[0]) # Ensure that the first value is a float, thus only considering values
        for itemIndex, item in enumerate(row):
            dataVal = float(item)
            if (dataVal == noDataVal) or (dataVal < 0):
                mapData[rowIndex][itemIndex]=0
            else:
                mapData[rowIndex][itemIndex]=dataVal
        
        rowIndex = rowIndex + 1
    except ValueError:
        pass

In [14]:
scaleDownGrid = False
count = 2

if scaleDownGrid:
    output = mapData.copy()
    for i in range(count):
        processingArray = output.copy()
        processingArray = processingArray[processingArray.shape[0]%3:]
        if(processingArray.shape[1]%3==1):
            np.delete(processingArray, [0], axis=1)
        if(processingArray.shape[1]%2==2):
            np.delete(processingArray, [0,1], axis=1)
        output = scaleDownArray(processingArray)
        print(i)

In [18]:
irradianceVals = ValueMap(gridSize, southWestPos, cellSize, noDataVal)
irradianceVals.setGridData(mapData)

In [22]:
irradianceVals.plot("gist_yarg")

In [21]:
irradianceVals.determineValue(33.8785317,151.1392979)

20627.688428999998

In [11]:
largestVal = 0
largestValIndex = []
for rowIndex, row in enumerate(mapData):
    for elemIndex, elem in enumerate(row):
        if(elem > largestVal) and (elem != 28384.754):
            largestVal = elem
            largestValIndex = [rowIndex, elemIndex]